In [11]:
%pip install pandas
%pip install statsmodels
%pip install chardet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 1.8 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import chardet
import warnings
warnings.filterwarnings("ignore")



file_path = 'data.csv' 

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']

df = pd.read_csv(file_path, encoding=encoding, sep=',')

if 'Год' not in df.columns or 'Номер недели' not in df.columns:
    raise KeyError("Отсутствуют необходимые столбцы 'Год' и 'Номер недели'.")


df['Дата'] = pd.to_datetime(df['Год'].astype(str) + '-W' + df['Номер недели'].astype(str) + '-1', format='%G-W%V-%u', errors='coerce')
df = df.dropna(subset=['Дата'])
df = df[df['Дата'].dt.year >= 2022]  
df.set_index('Дата', inplace=True)

grouped = df[['КТ', 'КТ с КУ 1 зона', 'КТ с КУ 2 и более зон', 'ММГ', 'МРТ', 'МРТ с КУ 1 зона', 'МРТ с КУ 2 и более зон', 'РГ', 'Флюорограф']]

best_params = {
    'КТ': {'order': (1, 1, 1), 'seasonal_order': (1, 1, 1, 52)},
    'КТ с КУ 1 зона': {'order': (1, 0, 1), 'seasonal_order': (1, 1, 0, 52)},
    'КТ с КУ 2 и более зон': {'order': (2, 1, 2), 'seasonal_order': (0, 1, 1, 52)},
    'ММГ': {'order': (1, 1, 0), 'seasonal_order': (1, 0, 1, 52)},
    'МРТ': {'order': (0, 1, 1), 'seasonal_order': (1, 1, 1, 52)},
    'МРТ с КУ 1 зона': {'order': (1, 0, 2), 'seasonal_order': (1, 1, 0, 52)},
    'МРТ с КУ 2 и более зон': {'order': (2, 1, 1), 'seasonal_order': (1, 1, 1, 52)},
    'РГ': {'order': (1, 1, 1), 'seasonal_order': (0, 1, 1, 52)},
    'Флюорограф': {'order': (1, 0, 1), 'seasonal_order': (1, 1, 0, 52)}
}

forecasts = {}

for column in grouped.columns:
    group = grouped[[column]].dropna()
    
    if len(group) < 10: 
        continue
    
    params = best_params.get(column, None)
    if params is None:
        continue
    
    order = params['order']
    seasonal_order = params['seasonal_order']
    
    try:
        model = SARIMAX(group[column], order=order, seasonal_order=seasonal_order)
        results = model.fit(disp=False)
        forecast = results.get_forecast(steps=4)
        
        forecasts[column] = forecast.predicted_mean
        
    except Exception as e:
        print(f"Не удалось построить модель для")

if forecasts:
    forecast_df = pd.DataFrame(forecasts)
    forecast_df = forecast_df.reset_index()
    forecast_df['index'] = pd.to_datetime(forecast_df['index'])  
    forecast_df = forecast_df[forecast_df['index'].dt.year > 2022]  
    forecast_df['Год'] = forecast_df['index'].dt.year
    forecast_df['Номер недели'] = forecast_df['index'].dt.isocalendar().week

    forecast_melted = forecast_df.melt(id_vars=['Год', 'Номер недели'], var_name='Тип исследования', value_name='Количество исследований')

    forecast_melted = forecast_melted.dropna(subset=['Количество исследований'])

    forecast_melted = forecast_melted[~forecast_melted['Количество исследований'].apply(lambda x: isinstance(x, pd.Timestamp))]

    forecast_melted = forecast_melted.sort_values(by=['Год', 'Номер недели']).reset_index(drop=True)

    forecast_list = forecast_melted.to_dict(orient='records')
    print(forecast_list)

[{'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'КТ', 'Количество исследований': 4776.994168850379}, {'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'КТ с КУ 1 зона', 'Количество исследований': 492.26746725212723}, {'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'КТ с КУ 2 и более зон', 'Количество исследований': 488.99296104085306}, {'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'ММГ', 'Количество исследований': 19909.236806572186}, {'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'МРТ', 'Количество исследований': 1845.8437925523208}, {'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'МРТ с КУ 1 зона', 'Количество исследований': 809.71996573569}, {'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'РГ', 'Количество исследований': 96984.4067991673}, {'Год': 2024, 'Номер недели': 5, 'Тип исследования': 'Флюорограф', 'Количество исследований': 10304.190090422202}, {'Год': 2024, 'Номер недели': 6, 'Тип исследования': 'КТ', 'Количество исследований': 3848.5906